## Paper Summary

### Basic Settings
- Network: $G = (V, E)$
- Feature representations: $f: V \rightarrow \mathcal{R}^d$
    - $d$: number of dimensions of feature representations
    - $\mathbf{f} \in \mathcal{R}^{n\times d} = [f(v_1), \dots, f(v_n)]^\top$
        - $v_i \in V$
        - $n = |V|$

- Network neighborhood: $N_S(v) \subset V$
    - $v \in V$
    - Sampling strategy: $S$

### Feature Learning Problem
- Objective function: maximum log conditional likelihood of $f$ with two assumptions
   $$\max_f \sum_{v\in V} \log{P(N_S(v) | f(v))} = \max_f \sum_{v\in V} \left[ -\log{\sum_{u \in V} e^{f(u)f(v)}} +  \sum_{v' \in N_S(v)} f(v') f(v) \right]$$

    - Conditional independence: $P(N_S(v) | f(v)) = \prod_{u' \in N_S(v)} P(v' | f(v))$
    - Softmax parameterization (with on symmetric feature space): $P(v' | f(v)) = \frac{e^{f(v')f(v)}}{\sum_{u \in V} e^{f(u)f(v)}}$
- Computation concerns
    1. $\sum_{u \in V} e^{f(u)f(v)}$ is expensive when the network is huge $\rightarrow$ Approximate it by negative sampling.
    2. Use stochastic gradient ascent to solve the right-handed side of the above objective function.

### Sampling Strategy
- The problem of sampling neighbors of a source node as a form of local search. 
- Aim to generate a node's ($v$) neighborhodd $N_S(v)$
- Example
   - Breadth-First Sampling (BFS)
   - Depth-First Sampling (DFS)
- Two common hypotheses
   - homophily:
     -  ***highly interconnected*** and belong to ***similar network*** clusters or communities should be embedded closely together.
     - macro-view of the neighborhood of every node
     - e.g., DFS
   - strctural equivalance:
      -  ***similar structure roles*** in networks should be embedded closely together.
      - not emphasize connectivity
      - microscopiv view of the neighborhood of every node
      - e.g., BFS

#### Random walks
  $$P(c_i = v' | c_{i-1} = v) = \left\{
    \begin{array}{ll}
    \frac{\pi_{vv'}}{Z},& (v, v') \in E\\
    0,& o.w.
    \end{array}
  \right.$$
   - $c_i$: $i$th node in the walk
   - $c_0 = u$ is the starting node
   - $\pi_{vv'}$: unnormalized transition probability between $v$ and $v'$
      - $\pi_{vv'}$ can be static edge weight $\omega_{vv'}$
   - $Z$: normalizing constant

- Second order random walks with search bias $\alpha_{pq} \in [0, 1]$. $\pi_{vv'} = \alpha_{pq}(t, v')\omega_{vv'}$
    $$\alpha_{pq}(t, v') = \left\{
        \begin{array}{ll}
            \frac{1}{p},&& d_{tv'} = 0\\
            1,&& d_{tv'} = 1\\
            \frac{1}{q},&& d_{tv'} = 2
        \end{array}
    \right.$$
    - $p, q$: parameters control how fast the walk explores and leaves the neighborhood of starting node $u$
        - return parameter $p$
           - if $p$ is low, the walk backtracks a step, i.e., keep the walk local close to $u$
        - in-out parameter $q$
            - if $q > 1$, the random walk is biased towards nodes close to node $t$ (aproximate BFS)
            - if $q < 1$, the random walk is inclined to visit nodes far away from node $t$ (aproximate DFS)

    - $d_{tv'} \in \{0, 1, 2\}$: shortest path distance between nodes $t$ and $v'$



#### Algorithm
- Three phases of `node2vec`
   1. preprocessing to compute transition probabilities
   2. generate random walks
   3. optimize the objective with SGD

- Algorithm
    - Input
        - Graph $G = (V, E, W)$
        - Dimensions (embedding size): $d$
        - Walk node size: $r$
        - Walk length $l$
        - Context size: $k$
        - Return parameter $p$
        - In-out parameter $q$
    - Output: $\mathbf{f} \in \mathcal{R}^{|V|\times d}$
    - Flow
      1. Cacluate $\pi = \text{PreporcessModifiedWeights}(G, p, q)$
      2. Create $G' = (V, E, \pi)$
      3. Create `walks = [node2vecWalk(G', v, l) for iter, v in zip(range(r), V)]`
      4. Compute `f = SGD(k, d, walks)`
    - Helper functions
        - Generate a random walk : `node2vecWalk(G = (V, E, \pi): graph, start_node, walk_length)`
            1. Initialize `walk = [start_node]`
            2. `for iter in range(walk_length):`
               1. `current_node = walk[-1]`
               2. `V_current = GetNeighbors(current_node, G)`
               3. `next_node = AliasSample(V_current, \pi)`
               4. `walk.append(next_node)`
            3. Return `walk`
         - Fetch neighbors by a node `GetNeighbors(node, graph)`
         - Sample a node by transition probability `AliasSample(node, \pi)`
       
       
 

